In [94]:
import cv2
import glob
import os
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from IPython.display import display
from skimage.draw import line



# read data

In [4]:
dataset_dir = "/home/vladushked/Documents/data/leolab_grasping_dataset/plastic/bottles"

background_path = os.path.join(dataset_dir, "background.png")

rgbs = glob.glob(os.path.join(dataset_dir, "pcd????r.png"))
depths = glob.glob(os.path.join(dataset_dir, "pcd????d.tiff"))
poss = glob.glob(os.path.join(dataset_dir, "pcd????cpos.txt"))
bboxes = glob.glob(os.path.join(dataset_dir, "pcd????bbox.txt"))
contours = glob.glob(os.path.join(dataset_dir, "pcd????cont.txt"))
meanlines = glob.glob(os.path.join(dataset_dir, "pcd????meanline.txt"))
minboxes = glob.glob(os.path.join(dataset_dir, "pcd????minbox.txt"))

rgbs.sort()
depths.sort()
poss.sort()
bboxes.sort()
contours.sort()
meanlines.sort()
minboxes.sort()

# Visualize

In [7]:
i = 0
colors = [list(np.random.random(size=3) * 256) for _ in range(20)]

In [72]:
rgb = cv2.imread(rgbs[i])
rgb = cv2.cvtColor(rgb, cv2.COLOR_BGR2RGB) 
print("RGB: ", os.path.split(rgbs[i])[-1])

depth = Image.open(depths[i])
# im.show()
depth = np.array(depth)

rows,cols = rgb.shape[:2]

pos_points = np.genfromtxt(poss[i], dtype="float")
pos_points = pos_points.reshape(-1, 4, 2).astype(int)

bbox = np.genfromtxt(bboxes[i], dtype="int")
contour = np.genfromtxt(contours[i], dtype="int")
[vx,vy,x,y] = np.genfromtxt(meanlines[i], dtype="float")
minbox = np.genfromtxt(minboxes[i], dtype="int")


RGB:  pcd0001r.png


In [82]:
def crop_bbox(img,bbox,margin=50):
    img = img.copy()
    [left, top] = bbox[0] - margin
    [right, bottom] = bbox[1] + margin
    return img[top:bottom, left:right]

In [83]:
def normalize(depth):
    copy = depth.copy()
    return (copy - np.min(copy)) / (np.max(copy) - np.min(copy))

In [112]:
rgb_copy = rgb.copy()
depth_copy = depth.copy()
normalized_depth = normalize(depth_copy)
cropped_normalized_depth = normalize(crop_bbox(depth_copy, bbox))

print("Depth max min: ", depth_copy.max(), depth_copy.min())
print("Depth normalized max min: ", normalized_depth.max(), normalized_depth.min())
print("Depth cropped normalized max min: ", cropped_normalized_depth.max(), cropped_normalized_depth.min())

for i, rect in enumerate(pos_points):
    cv2.drawContours(rgb_copy, [rect], 0, color=colors[i], thickness=4)

cv2.rectangle(rgb_copy,(bbox[0]),(bbox[1]),(0,255,0),2)

cv2.drawContours(rgb_copy, [contour], -1, (255, 0, 255), 2)
cv2.drawContours(rgb_copy,[minbox],0,(255,0,0),2, cv2.FILLED)

lefty = int((-x*vy/vx) + y)
righty = int(((cols-x)*vy/vx)+y)
cv2.line(rgb_copy,(cols-1,righty),(0,lefty),(255,255,0),2)

for pt in zip(*line(cols-1,righty,0,lefty)):
    if cv2.pointPolygonTest(contour, pt, False) == 0:  # If the point is on the contour
        cv2.circle(rgb_copy, pt, 2, (0, 0, 255), 2)


cropped_rgb = crop_bbox(rgb_copy, bbox)

fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(15,15))

axes[0,0].imshow(rgb)
axes[0,1].imshow(rgb_copy)
axes[1,0].imshow(depth)
axes[1,1].imshow(normalized_depth)
axes[2,0].imshow(cropped_rgb)
plot = axes[2,1].imshow(cropped_normalized_depth)
plt.colorbar(plot)

Depth max min:  31.039001 0.0
Depth normalized max min:  1.0 0.0
Depth cropped normalized max min:  1.0 0.0


error: OpenCV(4.5.4) :-1: error: (-5:Bad argument) in function 'pointPolygonTest'
> Overload resolution failed:
>  - Can't parse 'pt'. Sequence item with index 0 has a wrong type
>  - Can't parse 'pt'. Sequence item with index 0 has a wrong type


# generate grasps

In [106]:
[vx,vy,x,y]

[0.078102, -0.996945, 641.864075, 403.951477]

In [108]:
((cols-x)*vy/vx)+y

-7741.632755786932

In [97]:
line(cols-1,righty,0,lefty)

(array([1279, 1279, 1279, ...,    0,    0,    0]),
 array([-7741, -7740, -7739, ...,  8595,  8596,  8597]))

In [109]:
(cols-1,righty),(0,lefty)

((1279, -7741), (0, 8597))

In [121]:
pt = (1279, -7741)

-1.0

In [130]:
for pt in zip(*line(cols-1,righty,0,lefty)):
    print(pt)
    print(type(pt))
    print(cv2.pointPolygonTest(contour, (int(pt[0]), int(pt[1])), False))
    break

(1279, -7741)
<class 'tuple'>
-1.0
